In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, mean_squared_error, accuracy_score,recall_score,precision_score,f1_score
import xgboost as xgb
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [64]:
df = pd.read_excel("Raisin_Dataset.xlsx")

In [65]:
df
#feature engineering ?

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen
...,...,...,...,...,...,...,...,...
895,83248,430.077308,247.838695,0.817263,85839,0.668793,1129.072,Besni
896,87350,440.735698,259.293149,0.808629,90899,0.636476,1214.252,Besni
897,99657,431.706981,298.837323,0.721684,106264,0.741099,1292.828,Besni
898,93523,476.344094,254.176054,0.845739,97653,0.658798,1258.548,Besni


In [66]:
#Preprocessing
for col in df.columns[:-1]:
    df[col] = (df[col] - df[col].mean())/df[col].std()


In [67]:
for type in df["Class"].unique():
  df[f"Class_{type}"] = np.where(df["Class"] == type,1,0)

df.drop(["Class"], axis=1, inplace=True)

In [68]:
df

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class_Kecimen,Class_Besni
0,-0.007182,0.097523,-0.023945,0.422906,-0.015700,1.106128,0.066237,1,0
1,-0.324037,-0.208896,-0.229165,0.224352,-0.304079,-0.287617,-0.161163,1,0
2,0.078249,0.097704,0.236856,0.186136,0.062079,-1.157606,0.155858,1,0
3,-1.073689,-1.244359,-0.914765,-1.069028,-1.075567,0.001711,-1.175261,1,0
4,-0.215274,-0.678581,0.726949,-2.408488,-0.238491,1.744289,-0.338450,1,0
...,...,...,...,...,...,...,...,...,...
895,-0.116817,-0.007348,-0.133018,0.395494,-0.131155,-0.574454,-0.134549,0,1
896,-0.011644,0.084507,0.096122,0.299904,-0.007042,-1.178863,0.176595,0,1
897,0.303903,0.006697,0.887181,-0.662745,0.369835,0.777857,0.463615,0,1
898,0.146630,0.391383,-0.006243,0.710778,0.158622,-0.761381,0.338398,0,1


In [69]:
val = df.columns
labels = np.array(df[['Class_Kecimen','Class_Besni']])
#labels = labels.astype('int')
features_list = val
features = df.drop(['Class_Kecimen','Class_Besni'], axis = 1)
features = np.array(features)

In [70]:
x = df[val[:-2]]
y = df[['Class_Kecimen', 'Class_Besni']]
train_features, test_features, train_label, test_label = train_test_split(features, labels, test_size = 0.33, random_state = 42)

In [71]:
print(x)

         Area  MajorAxisLength  MinorAxisLength  Eccentricity  ConvexArea  \
0   -0.007182         0.097523        -0.023945      0.422906   -0.015700   
1   -0.324037        -0.208896        -0.229165      0.224352   -0.304079   
2    0.078249         0.097704         0.236856      0.186136    0.062079   
3   -1.073689        -1.244359        -0.914765     -1.069028   -1.075567   
4   -0.215274        -0.678581         0.726949     -2.408488   -0.238491   
..        ...              ...              ...           ...         ...   
895 -0.116817        -0.007348        -0.133018      0.395494   -0.131155   
896 -0.011644         0.084507         0.096122      0.299904   -0.007042   
897  0.303903         0.006697         0.887181     -0.662745    0.369835   
898  0.146630         0.391383        -0.006243      0.710778    0.158622   
899 -0.056282         0.699373        -0.784497      1.392886   -0.048789   

       Extent  Perimeter  
0    1.106128   0.066237  
1   -0.287617  -0.161

In [72]:
#Random Forest without grid search
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(features):   
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))

scores

[0.14166666666666666,
 0.15555555555555556,
 0.12222222222222222,
 0.14166666666666666,
 0.13333333333333333]

In [80]:
#Random Forest with grid search

rf = RandomForestClassifier()
    #rf.fit(train_features, train_label)

grid_values = { 
'n_estimators': [200, 500],
'max_features': ['auto', 'sqrt', 'log2'],
'max_depth' : [4,5,6,7,8],
'criterion' :['gini', 'entropy']
}
grid_clf_acc = GridSearchCV(rf, param_grid = grid_values,scoring = 'recall', cv=5)
grid_clf_acc.fit(train_features, train_label)
#[0.001,0.009,0.01,0.09,1,5,10,25]
#['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start']
y_pred = grid_clf_acc.predict(test_features)

print('Accuracy Score : ' + str(accuracy_score(test_label,y_pred)))
#print('Precision Score : ' + str(precision_score(test_label,y_pred)))
print('Recall Score : ' + str(recall_score(test_label,y_pred)))
print('F1 Score : ' + str(f1_score(test_label,y_pred)))

#Confusion matrix
print('Confusion Matrix : \n' + str(confusion_matrix(test_label,y_pred)))

c:\miniconda\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\miniconda\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\miniconda\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\miniconda\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "c:\miniconda\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "

Accuracy Score : 0.8417508417508418


ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

In [77]:
estimator = xgb.XGBRegressor(
    objective= 'reg:linear',
    nthread=4,
    seed=42
)

parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'f1_micro',
    n_jobs = 10,
    cv = 2,
    verbose=True
)

grid_search.fit(x, y)
grid_search.best_estimator_

y_pred = grid_search.predict(X_test)
    
scores.append(mean_squared_error(y_test, y_pred))

Fitting 2 folds for each of 96 candidates, totalling 192 fits
[17:25:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


c:\miniconda\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(
